# Extraction of features for different models of error-related brain activity and anxiety dimensions

### Imports

In [ ]:
import os
import sys
import re
import glob
import ast
import os.path as op
import pickle
import mne
import scipy
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp

from collections import defaultdict

from copy import deepcopy
import copy

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from scipy.integrate import simpson
from scipy.stats import boxcox
from scipy.stats import pearsonr

from autoreject import AutoReject

# from transformers import *

import warnings
warnings.filterwarnings("ignore")

---
## Load data

Loading EEG data and data from rumination questionnaire. By default create_df_data loads all info from given file but one can specify it by passing a list of desired labels from csv file.

In [ ]:
dir_path = os.path.dirname(os.path.abspath(""))

In [ ]:
tmin, tmax = -0.1, 0.6  # Start and end of the segments
signal_frequency = 256
random_state = 0

In [ ]:
def create_df_data(
    test_participants=False,
    test_epochs=False,
    info_filename=None,
    info="all",
    personal=True,
):
    """Loads data for all participants and create DataFrame with optional additional info from given .csv file.

    On default, loads a train set: chooses only 80% of participants
    and for each of them chooses 80% of epochs.
    It will choose them deterministically.

    Participants with less than 10 epochs per condition are rejected.

    If test_participants is set to True, it will load remaining 20% of participants.
    If test_epochs is set to True, it will load remaining 20% of epochs.
    Test epochs are chronologically after train epochs,
    because it reflects real usage (first callibration and then classification).

    Parameters
    ----------
    test_participants: bool
        whether load data for training or final testing.
        If true load participants data for testing.
    test_epochs: bool
        whether load data for training or final testing.
        If true load epochs of each participants data for testing.
    info_filename: String | None
        path to .csv file with additional data.
    info: array
        listed parameters from the info file to be loaded.
        if 'all', load all parameters
    personal: bool
        whether a model will be both trained and tested on epochs from one person
        if false, person's epochs aren't split into test and train


    Returns
    -------
    go_nogo_data_df : pandas.DataFrame

    """
    print(os.path.abspath(""))
    dir_path = os.path.dirname(os.path.abspath(""))
    print(dir_path)
    header_files_glob = os.path.join(dir_path, "data/responses_100_600_sonata/*.vhdr")
    header_files = glob.glob(header_files_glob)

    header_files = sorted(header_files)
    go_nogo_data_df = pd.DataFrame()

    # cut 20% of data for testing
    h_train, h_test = train_test_split(header_files, test_size=0.3, random_state=0)
    
    print(f"train size: {len(h_train)} ; test size: {len(h_test)}")

    if test_participants:
        header_files = h_test
    else:
        header_files = h_train

    for file in header_files:
        #  load eeg data for given participant
        participant_epochs = load_epochs_from_file(file)

        # and compute participant's id from file_name
        participant_id = re.match(r".*GNG-(\d+).*", file).group(1)

        error = participant_epochs["error_response"]._data
        correct = participant_epochs["correct_response"]._data

        # exclude those participants who have too few samples
        if len(error) < 5 or len(correct) < 5:
            # not enough data for this participant
            continue

        # construct dataframe for participant with: id|epoch_data|response_type|additional info...
        participant_df = create_df_from_epochs(
            participant_id, participant_epochs, info_filename, info
        )
        print(participant_id)
        go_nogo_data_df = go_nogo_data_df.append(participant_df, ignore_index=True)

    return go_nogo_data_df

In [ ]:
def create_df_from_epochs(id, participant_epochs, info_filename, info):
    """Create df for each participant. DF structure is like: {id: String ; epoch: epoch_data ; marker: 1.0|0.0}
    1.0 means correct and 0.0 means error response.
    Default info extracted form .csv file is 'Rumination Full Scale' and participants' ids.
    With this info df structure is like:
    {id: String ; epoch: epoch_data ; marker: 1.0|0.0 ; File: id ; 'Rumination Full Scale': int}

    Parameters
    ----------
    id: String
        participant's id extracted from filename
    correct: array
        correct responses' data
    error: array
        error responses' data
    info_filename: String
        path to .csv file with additional data.
    info: array
        listed parameters from the info file to be loaded.
        if 'all', load all parameters

    Returns
    -------
    participant_df : pandas.DataFrame

    """
    participant_df = pd.DataFrame()
    info_df = pd.DataFrame()

    # get additional info from file
    if info_filename is not None:
        if info == "all":
            rumination_df = pd.read_csv(info_filename, dtype={'Demo_kod': object})
        else:
            rumination_df = pd.read_csv(info_filename, usecols=["Demo_kod"] + info, dtype={'Demo_kod': object})
        info_df = (
            rumination_df.loc[rumination_df["Demo_kod"] == id]
            .reset_index()
            .drop("index", axis=1)
        )      
    epoch_df = pd.DataFrame({"id": [id], "epoch": [participant_epochs]}).join(
            info_df
        )
    participant_df = participant_df.append(epoch_df, ignore_index=True)

    return participant_df

In [ ]:
def load_epochs_from_file(file, reject_bad_segments="auto", mask=None):
    """Load epochs from a header file.

    Args:
        file: path to a header file (.vhdr)
        reject_bad_segments: 'auto' means that bad segments are rejected automatically.

    Returns:
        mne Epochs

    """
    # Import the BrainVision data into an MNE Raw object
    raw = mne.io.read_raw_brainvision(file)

    # Construct annotation filename
    annot_file = file[:-4] + "vmrk"

    # Read in the event information as MNE annotations
    annotations = mne.read_annotations(annot_file)

    # Add the annotations to our raw object so we can use them with the data
    raw.set_annotations(annotations)

    # Map with response markers only
    # event_dict = {
    #     "Stimulus/RE*ex*1_n*1_c_1*R*FB": 10004,
    #     "Stimulus/RE*ex*1_n*1_c_1*R*FG": 10005,
    #     "Stimulus/RE*ex*1_n*1_c_2*R": 10006,
    #     "Stimulus/RE*ex*1_n*2_c_1*R": 10007,
    #     "Stimulus/RE*ex*2_n*1_c_1*R": 10008,
    #     "Stimulus/RE*ex*2_n*2_c_1*R*FB": 10009,
    #     "Stimulus/RE*ex*2_n*2_c_1*R*FG": 10010,
    #     "Stimulus/RE*ex*2_n*2_c_2*R": 10011,
    # }
    
    event_dict = {
        'Stimulus/RE*ex*1_n*1_c_1*R*FB': 10003,
        'Stimulus/RE*ex*1_n*1_c_1*R*FG': 10004,
        'Stimulus/RE*ex*1_n*1_c_2*R': 10005,
        'Stimulus/RE*ex*1_n*2_c_1*R': 10006,
        'Stimulus/RE*ex*2_n*1_c_1*R': 10007,
        'Stimulus/RE*ex*2_n*2_c_1*R*FB': 10008,
        'Stimulus/RE*ex*2_n*2_c_1*R*FG': 10009,
        'Stimulus/RE*ex*2_n*2_c_2*R': 10010,
    }

    # Map for merged correct/error response markers
    merged_event_dict = {"correct_response": 0, "error_response": 1}

    # Reconstruct the original events from Raw object
    events, event_ids = mne.events_from_annotations(raw, event_id=event_dict)

    # Merge correct/error response events
    merged_events = mne.merge_events(
        events,
        [10003, 10004, 10008, 10009],
        merged_event_dict["correct_response"],
        replace_events=True,
    )
    merged_events = mne.merge_events(
        merged_events,
        [10005, 10006, 10007, 10010],
        merged_event_dict["error_response"],
        replace_events=True,
    )

    epochs = []
    bads = []
    this_reject_by_annotation = False
    
    # Read epochs
    epochs = mne.Epochs(
        raw=raw,
        events=merged_events,
        event_id=merged_event_dict,
        tmin=tmin,
        tmax=tmax,
        baseline=None,
        reject_by_annotation=this_reject_by_annotation,
        preload=True,
    )
    
    ar = AutoReject(random_state=random_state, n_jobs=10, verbose=0)
    epochs_ar, reject_log = ar.fit_transform(epochs, return_log=True)

    # reject trials bad at Fz
    idx = reject_log.ch_names.index('Fz')
    mask = (reject_log.labels[:,idx] == 1)
    epochs_ar.drop(mask)
    
    print(f'Reject {len(np.where(mask == True)[0])} trials bad at Fz')    
    
    return epochs_ar

#### Read the data

In [ ]:
def change_column_names(data_df):
    columns_dict = {
        "16-Rumination Full Scale": "RRQ", # mean
        "05-DASS-21 Anxiety scale": "DASS-21 Anx", # mean
        "05-DASS-21 Stress scale": "DASS-21 Stress", # mean
        "05-DASS-21 Depression scale": "DASS-21 Dep", # mean
        "04-STAI Trait MEAN": "STAI-T_M", # mean
        "04-STAI STATE 1-2 DIFFERENCE": "STAI-S Diff", # sum
        "number_error" : "uninhibited response", # sum
        "number_inhibited" : "inhibited response", # sum
        "04-STAI Trait SUM": "STAI-T", # sum
        "07-BIS": "BIS", # mean
        "14-Obsessive-Compulsive WASHING": "WASH", # mean
        "14-Obsessive-Compulsive OBSESSING": "OBSESS", # mean
        "14-Obsessive-Compulsive HOARDING": "HOARD", # mean
        "14-Obsessive-Compulsive ORDERING": "ORD", # mean
        "14-Obsessive-Compulsive CHECKING": "CHECK", # mean
        "14-Obsessive-Compulsive NEUTRALIZING": "NEU", # mean
        "18-Thought Suppression Inventory": "WBSI", # mean
        "28-Intolerance of Uncertainty - Prospective Anxiety": "IUS-P", # mean
        "28-Intolerance of Uncertainty - Inhibitory Anxiety": "IUS-I", # mean
        "06-Self-Esteem Scale_SES Rosenberga": "SES", # mean
        "07-BAS Dzialanie": 'BAS_D', # mean
        "07-BAS Poszukiwanie przyjemnosci": 'BAS_PRZY', # mean
        "07-BAS Wrazliwosc na nagrode": 'BAS_NAG', # mean
        "22-Nonforgiveness - Full Scale": 'NONFOR',
        "27-Indecisiveness Scale_Frost": 'INDEC_F', # mean
        "03-SP (Punishment Sensitivity)": 'PUN', # sum
        "03-SR (Reward Sensitivity)": 'REW', # sum
        "15-Obsessional Beliefs - Inflated responsibility for harm": 'HARM', # mean
        "15-Obsessional Beliefs - Importance/Control of Thoughts": 'T-CTR', # mean
        "15-Obsessional Beliefs - Overestimation of threat": "OT", # mean
        "15-Obsessional Beliefs - Perfectionism/ Intolerance of uncertainty": 'OB_PERF', # mean
        "17-Perfectionism CMDA": 'CMDA', # mean
        "17-Perfect PS-Personal Standards (7 items mean)" : 'PS', # mean
        "19-Guilt sensitivity": 'G_SE', # mean
        "31-NFC Nietolerancja wieloznaczności-FULL": 'AMB', # mean
        "31-NFC Preferowanie przewidywalności-FULL": 'PRED', # mean
        "32-High standards from Maximization Full Scale" : 'STAND',   # mean
        "Wiek": "Age",
        "Płeć": "Sex",
        "Ręczność": "Handness",    

        #######
        "Rumination Full Scale": "RRQ",
        "DASS-21 Anxiety scale 0-SUM": "DASS-21 Anx", # sum
        "DASS-21 Stress scale 0-SUM": "DASS-21 Stress", # sum
        "DASS-21 Depression scale 0-SUM": "DASS-21 Dep", # sum
        "number_error": "uninhibited response", # sum
        "number_inhibited":  "inhibited response", # sum
        "STAI STATE 1-2 DIFFERENCE": "STAI-S Diff", # sum
        "STAI Trait SUM": "STAI-T", # sum
        "BIS": "BIS", # mean
        "Obsessive-Compulsive WASHING": "WASH", # mean
        "Obsessive-Compulsive OBSESSING": "OBSESS", # mean
        "Obsessive-Compulsive HOARDING": "HOARD", # mean
        "Obsessive-Compulsive ORDERING": "ORD", # mean
        "Obsessive-Compulsive CHECKING": "CHECK", # mean
        "Obsessive-Compulsive NEUTRALIZING": "NEU", # mean
        "Thought Suppression Inventory": "WBSI", # mean
        "Intolerance of Uncertainty - Prospective Anxiety": "IUS-P", # mean
        "Intolerance of Uncertainty - Inhibitory Anxiety": "IUS-I", # mean
        "Self-Esteem Scale_SES Rosenberga MEAN": "SES", # mean
        "BAS Dzialanie": 'BAS_D', # mean # drive
        "BAS Poszukiwanie przyjemnosci": 'BAS_PRZY', # mean # fun seeking
        "BAS Wrazliwosc na nagrode": 'BAS_NAG', # mean # responsivness
        "Indecisiveness Scale_Frost": 'INDEC_F', # mean
        "SP (Punishment Sensitivity)": 'PUN', # sum
        "SR (Reward Sensitivity)": 'REW', # sum
        "Obsessional Beliefs - Inflated responsibility for harm": 'HARM', # mean
        "Obsessional Beliefs - Importance/Control of Thoughts": 'T-CTR', # mean
        "Obsessional Beliefs - Overestimation of threat": "OT", # mean
        "Obsessional Beliefs - Perfectionism/ Intolerance of uncertainty": 'OB_PERF', # mean
        "Perfect PS-Personal Standards (7 items mean)" : 'PS', # mean
        "Guilt sensitivity": 'G_SE', # mean
        "Nietolerancja wieloznaczności-FULL": 'AMB', # mean
        "Preferowanie przewidywalności-FULL": 'PRED', # mean
        "High standards from Maximization Full Scale" : 'STAND',   # mean
    }

    data_df = data_df.rename(columns=columns_dict)
    return data_df

- read Study 1 train data

In [ ]:
# %%capture
df_name = "GNG_autoreject_3_5"
pickled_data_filename = "../data/" + df_name + ".pkl"
info_filename = "../data/scales/all_scales_with_rt.csv"


# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_train_opus_df = pd.read_pickle(pickled_data_filename)
    epochs_train_opus_df.name = df_name
    epochs_train_opus_df = change_column_names(epochs_train_opus_df)
    print("Done")
    pass
else:
    print("Pickled file not found. Loading data...")
    epochs_train_opus_df = create_df_data(
        test_participants=False, info="all", personal=False, info_filename=info_filename
    )
    epochs_train_opus_df.name = df_name
    epochs_train_opus_df = change_column_names(epochs_train_opus_df)
    
    # save loaded data into a pickle file
    epochs_train_opus_df.to_pickle("../data/" + epochs_train_opus_df.name + ".pkl")
    print("Done. Pickle file created")

In [ ]:
epochs_train_opus_df.shape

In [ ]:
epochs_train_opus_df['STAI-T'] = epochs_train_opus_df['STAI-T'].apply(lambda x: x/20)

- read Study 2 train data

In [ ]:
# %%capture
df_name = "GNG_autoreject_sonata_3_5_stai"
# df_name = "GNG_reject_auto_3-5"
pickled_data_filename = "../data/" + df_name + ".pkl"
info_filename = "../data/scales/Sonata_scales.csv"


# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_train_sonata_df = pd.read_pickle(pickled_data_filename)
    epochs_train_sonata_df.name = df_name
    epochs_train_sonata_df = change_column_names(epochs_train_sonata_df)
    print("Done")
    pass
else:
    print("Pickled file not found. Loading data...")
    epochs_train_sonata_df = create_df_data(
        test_participants=False, info="all", personal=False, info_filename=info_filename
    )
    epochs_train_sonata_df.name = df_name
    epochs_train_sonata_df = change_column_names(epochs_train_sonata_df)
    
    # save loaded data into a pickle file
    epochs_train_sonata_df.to_pickle("../data/" + epochs_train_sonata_df.name + ".pkl")
    print("Done. Pickle file created")

In [ ]:
epochs_train_sonata_df.shape

Refine some data from questionnaries to reconcile data from two datasets

In [ ]:
epochs_train_sonata_df.loc[epochs_train_sonata_df['Sex'] == 'Osoba niebinarna', 'Sex'] = 0.5

In [ ]:
# normalize DASS-21 subscales scores to mean
epochs_train_sonata_df['DASS-21 Stress'] = epochs_train_sonata_df['DASS-21 Stress'].apply(lambda x: (x + 7)/7)
epochs_train_sonata_df['DASS-21 Anx'] = epochs_train_sonata_df['DASS-21 Anx'].apply(lambda x: (x + 7)/7)
epochs_train_sonata_df['DASS-21 Dep'] = epochs_train_sonata_df['DASS-21 Dep'].apply(lambda x: (x + 7)/7)

In [ ]:
# normalize STAI state scale to mean (divide by amount of items in subscale)
epochs_train_sonata_df['STAI-T'] = epochs_train_sonata_df['STAI-T'].apply(lambda x: x/20)

In [ ]:
# fill missing value in STAI-S Diff score
epochs_train_sonata_df.loc[epochs_train_sonata_df['STAI-S Diff'] == 'None', 'STAI-S Diff'] = None
epochs_train_sonata_df['STAI-S Diff'] = np.array(epochs_train_sonata_df['STAI-S Diff'].to_list()).astype(np.float) 

In [ ]:
epochs_train_sonata_df['STAI-S Diff'] = epochs_train_sonata_df['STAI-S Diff'].fillna(epochs_train_sonata_df['STAI-S Diff'].mean())

#### Read data for external testing

- read Study 1 test

In [ ]:
# %%capture
df_name = "GNG_autoreject_3_5_test_performance"
pickled_data_filename = "../data/" + df_name + ".pkl"
info_filename = "../data/scales/all_scales_with_rt.csv"


# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_test_opus_df = pd.read_pickle(pickled_data_filename)
    epochs_test_opus_df.name = df_name
    epochs_test_opus_df = change_column_names(epochs_test_opus_df)
    print("Done")
else:
    print("Pickled file not found. Loading data...")
    epochs_test_opus_df = create_df_data(
        test_participants=True, info="all", personal=False, info_filename=info_filename
    )
    epochs_test_opus_df.name = df_name
    epochs_test_opus_df = change_column_names(epochs_test_opus_df)
    
    # save loaded data into a pickle file
    epochs_test_opus_df.to_pickle("../data/" + epochs_test_opus_df.name + ".pkl")
    print("Done. Pickle file created")

In [ ]:
epochs_test_opus_df.shape

In [ ]:
epochs_test_opus_df['STAI-T'] = epochs_test_opus_df['STAI-T'].apply(lambda x: x/20)

- read Study 2 test

In [ ]:
# %%capture
df_name = "GNG_autoreject_sonata_3_5_test_stai"
pickled_data_filename = "../data/" + df_name + ".pkl"
info_filename = "../data/scales/Sonata_scales.csv"


# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_test_sonata_df = pd.read_pickle(pickled_data_filename)
    epochs_test_sonata_df.name = df_name
    epochs_test_sonata_df = change_column_names(epochs_test_sonata_df)
    print("Done")
else:
    print("Pickled file not found. Loading data...")
    epochs_test_sonata_df = create_df_data(
        test_participants=True, info="all", personal=False, info_filename=info_filename
    )
    epochs_test_sonata_df.name = df_name
    epochs_test_sonata_df = change_column_names(epochs_test_sonata_df)
    
    # save loaded data into a pickle file
    epochs_test_sonata_df.to_pickle("../data/" + epochs_test_sonata_df.name + ".pkl")
    print("Done. Pickle file created")

In [ ]:
epochs_test_sonata_df.shape

Refine some data from questionnaries to reconcile data from two datasets

In [ ]:
epochs_test_sonata_df.loc[epochs_test_sonata_df['Sex'] == 'Osoba niebinarna', 'Sex'] = 0.5

In [ ]:
# normalize DASS-21 subscales scores to mean
epochs_test_sonata_df['DASS-21 Stress'] = epochs_test_sonata_df['DASS-21 Stress'].apply(lambda x: (x + 7)/7)
epochs_test_sonata_df['DASS-21 Anx'] = epochs_test_sonata_df['DASS-21 Anx'].apply(lambda x: (x + 7)/7)
epochs_test_sonata_df['DASS-21 Dep'] = epochs_test_sonata_df['DASS-21 Dep'].apply(lambda x: (x + 7)/7)

In [ ]:
# normalize STAI state scale to mean (divide by amount of items in subscale)
epochs_test_sonata_df['STAI-T'] = epochs_test_sonata_df['STAI-T'].apply(lambda x: x/20)

In [ ]:
# fill missing value in STAI-S Diff score
epochs_test_sonata_df.loc[epochs_test_sonata_df['STAI-S Diff'] == 'None', 'STAI-S Diff'] = None
epochs_test_sonata_df['STAI-S Diff'] = np.array(epochs_test_sonata_df['STAI-S Diff'].to_list()).astype(np.float) 

In [ ]:
epochs_test_sonata_df['STAI-S Diff'] = epochs_test_sonata_df['STAI-S Diff'].fillna(epochs_test_sonata_df['STAI-S Diff'].mean())

## Merge dataframes

In [ ]:
opus_columns_list = epochs_train_opus_df.columns.to_list()
sonata_columns_list = epochs_train_sonata_df.columns.to_list()

columns = list(set(opus_columns_list) & set(sonata_columns_list))

In [ ]:
epochs_train_sonata_df = epochs_train_sonata_df[columns]
epochs_train_opus_df = epochs_train_opus_df[columns]

epochs_test_sonata_df = epochs_test_sonata_df[columns]
epochs_test_opus_df = epochs_test_opus_df[columns]

Create train and test data

In [ ]:
epochs_train_df = pd.concat([epochs_train_sonata_df, epochs_train_opus_df], ignore_index=True)
epochs_test_df = pd.concat([epochs_test_sonata_df, epochs_test_opus_df], ignore_index=True)

#### Chose dataset

In [ ]:
test = False
dataset = 'test' if test else 'train'

In [ ]:
epochs_df = epochs_train_df if not test else epochs_test_df

### Remove subject with low signal consistency

In [ ]:
within_consistency_ern_df = pd.read_csv(f'../new_results/consistency_tests/{dataset}/ern_amplitude_within.csv')
internal_consistency_ern_df = pd.read_csv(f'../new_results/consistency_tests/{dataset}/ern_amplitude_consistency.csv')
within_consistency_crn_df = pd.read_csv(f'../new_results/consistency_tests/{dataset}/crn_amplitude_within.csv')
internal_consistency_crn_df = pd.read_csv(f'../new_results/consistency_tests/{dataset}/crn_amplitude_consistency.csv')


consistency_ern_df = within_consistency_ern_df.merge(internal_consistency_ern_df, on ='Unnamed: 0')
consistency_ern_df = consistency_ern_df[consistency_ern_df['pipeline_x'] == 'Fz']

consistency_crn_df = within_consistency_crn_df.merge(internal_consistency_crn_df, on ='Unnamed: 0')
consistency_crn_df = consistency_crn_df[consistency_crn_df['pipeline_x'] == 'Fz']

consistency_df = consistency_ern_df.merge(consistency_crn_df, on='id', suffixes=('_ern', '_crn'))
# consistency_df = consistency_df.rename(columns={'id_x': 'id'})
consistency_df.head()

Plot relationship between consistency and number of trials

In [ ]:
fig = sns.scatterplot(
    consistency_df,
    x='num_trials_ern',
    y='internal_variance_ern'
)

plt.axhline(y=.7)

Filter data

In [ ]:
exclude_consistency = True

In [ ]:
threshold_consistency = 0.7

if exclude_consistency:
    epochs_df = epochs_df.merge(consistency_df[['internal_variance_ern', 'internal_variance_crn', 'id']], on='id')
    epochs_df = epochs_df[
        (epochs_df['internal_variance_ern'] >= threshold_consistency) &
        (epochs_df['internal_variance_crn'] >= threshold_consistency)
    ]
    
epochs_df

## Explore data

- average number of responses in the train and test sets

In [ ]:
error_len = epochs_df['epoch'].map(lambda x: len(x['error_response'].get_data())).to_numpy()
correct_len = epochs_df['epoch'].map(lambda x: len(x['correct_response'].get_data())).to_numpy()

In [ ]:
print(f"AVG number of incorrect responses in the train set: {np.mean(error_len)} (SD={np.std(error_len)})")
print(f"AVG number of correct responses in the train set: {np.mean(correct_len)} (SD={np.std(correct_len)})")

In [ ]:
error_len = epochs_df['epoch'].map(lambda x: len(x['error_response'].get_data())).to_numpy()
correct_len = epochs_df['epoch'].map(lambda x: len(x['correct_response'].get_data())).to_numpy()

In [ ]:
print(f"AVG number of incorrect responses in the test set: {np.mean(error_len)} (SD={np.std(error_len)})")
print(f"AVG number of correct responses in the test set: {np.mean(correct_len)} (SD={np.std(correct_len)})")

- difference in the 0-100 ms window between conditions

In [ ]:
roi = ["Fz"]

In [ ]:
epochs_df_copy = pd.DataFrame(copy.deepcopy(epochs_df.to_dict()))

ern_pipeline = Pipeline([
    ("channels_extraction",PickChannels(channels_list=roi)),
    ("trim", EpochTrim(tmin=0, tmax=0.1)),
    ("average", Evoked()),
    ('extract_averaged_data', ExtractData()),
    ("mean_amplitude", AverageSignal()),
    
]).fit(epochs_df_copy)

preprocessed_X_ern_train = ern_pipeline.transform(epochs_df_copy)

In [ ]:
epochs_df_copy = pd.DataFrame(copy.deepcopy(epochs_df.to_dict()))

crn_pipeline = Pipeline([
   ("channels_extraction",PickChannels(channels_list=roi)),
    ("trim", EpochTrim(tmin=0, tmax=0.1)),
    ("average", Evoked(condition='correct_response')),
    ('extract_averaged_data', ExtractData()),
    ("mean_amplitude", AverageSignal()),
    
]).fit(epochs_df_copy)

preprocessed_X_crn_train = crn_pipeline.transform(epochs_df_copy)

In [ ]:
epochs_df_copy = pd.DataFrame(copy.deepcopy(epochs_df.to_dict()))

ern_pipeline = Pipeline([
    ("channels_extraction",PickChannels(channels_list=roi)),
    ("trim", EpochTrim(tmin=0, tmax=0.1)),
    ("average", Evoked()),
    ('extract_averaged_data', ExtractData()),
    ("mean_amplitude", AverageSignal()),
    
]).fit(epochs_df_copy)

preprocessed_X_ern_test = ern_pipeline.transform(epochs_df_copy)

In [ ]:
epochs_df_copy = pd.DataFrame(copy.deepcopy(epochs_df.to_dict()))

crn_pipeline = Pipeline([
   ("channels_extraction",PickChannels(channels_list=roi)),
    ("trim", EpochTrim(tmin=0, tmax=0.1)),
    ("average", Evoked(condition='correct_response')),
    ('extract_averaged_data', ExtractData()),
    ("mean_amplitude", AverageSignal()),
    
]).fit(epochs_df_copy)

preprocessed_X_crn_test = crn_pipeline.transform(epochs_df_copy)

In [ ]:
scipy.stats.ttest_rel(preprocessed_X_ern_train, preprocessed_X_crn_train)

In [ ]:
scipy.stats.ttest_rel(preprocessed_X_ern_test, preprocessed_X_crn_test)

- Visualization of ERPs

In [ ]:
chs = [
'Fp1',
 'AF7',
 'AF3',
 'F1',
 'F3',
 'F5',
 'F7',
 'FT7',
 'FC5',
 'FC3',
 'FC1',
 'C1',
 'C3',
 'C5',
 'T7',
 'TP7',
 'CP5',
 'CP3',
 'CP1',
 'P1',
 'P3',
 'P5',
 'P7',
 'P9',
 'PO7',
 'PO3',
 'O1',
 'Iz',
 'Oz',
 'POz',
 'Pz',
 'CPz',
 'Fpz',
 'Fp2',
 'AF8',
 'AF4',
 'AFz',
 'Fz',
 'F2',
 'F4',
 'F6',
 'F8',
 'FT8',
 'FC6',
 'FC4',
 'FC2',
 'FCz',
 'C2',
 'C4',
 'C6',
 'T8',
 'TP8',
 'CP6',
 'CP4',
 'CP2',
 'P2',
 'P4',
 'P6',
 'P8',
 'P10',
 'PO8',
 'PO4',
 'O2',
 'Cz']

In [ ]:
channel = chs
epochs_chs = epochs_df['epoch'].map(lambda x: x.copy().pick(channel)).to_list()
all_epochs = mne.concatenate_epochs(epochs_chs)

error_evokes = all_epochs['error_response'].average()
correct_evokes = all_epochs['correct_response'].average()

ERN and CRN waveforms

In [ ]:
# plot parameters
cm = 1/2.54
dpi = 500

plt.rc('font', size=3)
sns.set_style("ticks")
plt.rcParams['figure.dpi'] = dpi
plt.rcParams['ytick.labelsize'] = 5
plt.rcParams['xtick.labelsize'] = 5
plt.rcParams['axes.labelsize'] = 6
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.5
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.spines.bottom']= True
plt.rcParams['axes.spines.left']= True
plt.rcParams['axes.spines.right'] = True
plt.rcParams['axes.spines.top'] = True
plt.rcParams['axes.titlesize'] = 6
plt.rcParams['figure.frameon'] = True
plt.rc('legend',fontsize=4)
plt.rcParams['axes.titlepad'] = 0.2
plt.rcParams['axes.labelpad'] = 2
plt.rcParams['axes.titlepad'] = 0.2

# create canvas
fig = plt.figure(figsize=(7*cm, 3*cm))
ax = fig.add_axes([0,0,1,1], zorder=-1)

# add axes from topo maps
topo_axis = fig.add_axes([0.6,0.7,0.4,0.4], anchor='NE', zorder=2)
topo_axis_cbar = fig.add_axes([0.1,0.12,10,200], anchor='NE', zorder=10)

# plot topomap
topo_fig = error_evokes.plot_topomap(
    times=.05, 
    average = .103,
    res=300,
    # size=0.5,
    show=False,
    axes=[topo_axis,topo_axis_cbar],
    contours=6,
    extrapolate='head',
    time_format="%01d",
    time_unit='ms',
    sensors=True,
    vlim=(-10,10)
)

topo_axis_cbar.remove()
topo_axis.set_title('', size=7, pad=4) # Title
plt.setp(topo_axis.lines , linewidth=0.3)

# Define the colors for each condition
colors = {
    'correct': '#c94641',
    'error': '#2166ac',
}

# plot condition wavelines
fig_list = mne.viz.plot_compare_evokeds(
    dict(correct=correct_evokes, error=error_evokes),
    picks='Fz',
    legend='upper left',
    show_sensors=False,
    time_unit='ms',
    show=False,
    axes=ax,
    colors=colors,
    ylim=dict(eeg=[-15, 18]),
    invert_y=True,
)
# change the linewidth of the sensors
for coll in topo_axis.collections:
    if coll.get_label() == '_child10':
        coll.set_linewidth(.3)  
        break

fig_list[0].axes[0].tick_params(axis='x', direction='in', length=2.5, width=1)
fig_list[0].axes[0].tick_params(axis='y', direction='out', length=2.5, width=1)

plt.show()

# fig.savefig(f'waveforms', bbox_inches='tight', pad_inches=0.01)

With CRN

In [ ]:
diff = error_evokes.get_data() - correct_evokes.get_data()
diff_evokeds = mne.EvokedArray(diff, info = error_evokes.info, tmin = error_evokes.tmin,)

In [ ]:
# plot parameters
cm = 1/2.54
dpi = 500

plt.rc('font', size=3)
sns.set_style("ticks")
plt.rcParams['figure.dpi'] = dpi
plt.rcParams['ytick.labelsize'] = 5
plt.rcParams['xtick.labelsize'] = 5
plt.rcParams['axes.labelsize'] = 6
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.5
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.spines.bottom']= True
plt.rcParams['axes.spines.left']= True
plt.rcParams['axes.spines.right'] = True
plt.rcParams['axes.spines.top'] = True
plt.rcParams['axes.titlesize'] = 6
plt.rcParams['figure.frameon'] = True
plt.rc('legend',fontsize=4)
plt.rcParams['axes.titlepad'] = 0.2
plt.rcParams['axes.labelpad'] = 2
plt.rcParams['axes.titlepad'] = 0.2

# create canvas
fig = plt.figure(figsize=(7*cm, 3*cm))
ax = fig.add_axes([0,0,1,1], zorder=-1)

# add axes from topo maps
topo_axis = fig.add_axes([0.6,0.7,0.35,0.35], anchor='NE', zorder=2)
topo_axis_cbar = fig.add_axes([1,.64,.02,0.37], anchor='NE', zorder=10)

# plot topomap
topo_fig = error_evokes.plot_topomap(
    times=.05, 
    average = .103,
    res=300,
    # size=0.5,
    show=False,
    axes=[topo_axis,topo_axis_cbar],
    contours=6,
    extrapolate='head',
    time_format="%01d",
    time_unit='ms',
    sensors=True,
    vlim=(-5,5)
)

topo_axis.set_title('', size=7, pad=4) # Title
plt.setp(topo_axis.lines , linewidth=0.3)

# change the linewidth of the sensors
for coll in topo_axis.collections:
    if coll.get_label() == '_child10':
        coll.set_linewidth(.3)  
        break
        
# add axes from topo maps
topo_axis_diff = fig.add_axes([0.6,0.05,0.35,0.35], anchor='NE', zorder=2)
topo_axis_cbar_diff = fig.add_axes([0.1,10,10,200], anchor='NE', zorder=10)

topo_axis_cbar.tick_params(labelsize=4)
topo_axis_cbar.set_yticks([-6, -3, 0, 3, 6]) 

# plot topomap
topo_fig = correct_evokes.plot_topomap(
    times=.05, 
    average = .103,
    res=300,
    # size=0.5,
    show=False,
    axes=[topo_axis_diff,topo_axis_cbar_diff],
    contours=6,
    extrapolate='head',
    time_format="%01d",
    time_unit='ms',
    sensors=True,
    vlim=(-5,5)
)

topo_axis_cbar_diff.remove()
topo_axis_diff.set_title('', size=7, pad=4) # Title
plt.setp(topo_axis_diff.lines , linewidth=0.3)

# change the linewidth of the sensors
for coll in topo_axis_diff.collections:
    if coll.get_label() == '_child10':
        coll.set_linewidth(.3)  
        break

# Define the colors for each condition
colors = {
    'correct': '#c94641',
    'error': '#2166ac',
    'difference': 'gray'
}

# plot condition wavelines
fig_list = mne.viz.plot_compare_evokeds(
    dict(correct=correct_evokes, error=error_evokes, difference=diff_evokeds),
    picks='Fz',
    legend='upper left',
    show_sensors=False,
    time_unit='ms',
    show=False,
    axes=ax,
    colors=colors,
    ylim=dict(eeg=[-15, 18]),
    invert_y=True,
    styles={'correct': {"linestyle": '-'}, 'error': {"linestyle": '-'}, "difference": {"linestyle": '--'}},
)


fig_list[0].axes[0].tick_params(axis='x', direction='in', length=2.5, width=1)
fig_list[0].axes[0].tick_params(axis='y', direction='out', length=2.5, width=1)

plt.show()

fig.savefig(f'waveforms_ern_crn', bbox_inches='tight', pad_inches=0.01)

## Extract features

#### Extract EEG features

1. Amplitude

- ROI: Fz
- time window: 0 - 100 ms
- mean amplitude

- ERN

In [ ]:
roi = ["Fz"]

In [ ]:
epochs_df_copy = pd.DataFrame(copy.deepcopy(epochs_df.to_dict()))

ern_pipeline = Pipeline([
    ("channels_extraction",PickChannels(channels_list=roi)),
    ("trim", EpochTrim(tmin=0, tmax=0.1)),
    ("average", Evoked()),
    ('extract_averaged_data', ExtractData()),
    ("mean_amplitude", AverageSignal()),
    
]).fit(epochs_df_copy)

preprocessed_X_ern = ern_pipeline.transform(epochs_df_copy)
preprocessed_X_ern = preprocessed_X_ern.reshape(preprocessed_X_ern.shape[0], -1)

In [ ]:
preprocessed_X_ern.shape

- CRN

In [ ]:
roi = ["Fz"]

In [ ]:
epochs_df_copy = pd.DataFrame(copy.deepcopy(epochs_df.to_dict()))

crn_pipeline = Pipeline([
   ("channels_extraction",PickChannels(channels_list=roi)),
    ("trim", EpochTrim(tmin=0, tmax=0.1)),
    ("average", Evoked(condition='correct_response')),
    ('extract_averaged_data', ExtractData()),
    ("mean_amplitude", AverageSignal()),
    
]).fit(epochs_df_copy)

preprocessed_X_crn = crn_pipeline.transform(epochs_df_copy)
preprocessed_X_crn = preprocessed_X_crn.reshape(preprocessed_X_crn.shape[0], -1)

In [ ]:
preprocessed_X_crn.shape

2. Latency: fractional area latency

In [ ]:
def fractional_negative_area_latency(evoked, fraction=0.5, tmin=0.0, tmax=0.5, threshold = 0.0):
    subject_data = evoked.get_data()
    x = np.linspace(tmin, tmax, subject_data.shape[-1])
    y = subject_data.flatten()
    
    # get only negative part of signal
    y_negative = [abs(y_item) if y_item < threshold else 0 for y_item in y]
    
    # calculate area under the signal
    area = abs(simpson(y_negative, x))
    
    if area != 0.0:
        fractional_area = area * fraction
    
        # search for latency point (x) which split area according to fraction provided 
        current_area = 0
        fractional_area_index = 0
        i = 2
        while abs(simpson(y_negative[:i], x[:i])) <= fractional_area:
            current_area = abs(simpson(y_negative[:i], x[:i]))
            fractional_area_index = i
            i+=1
        return (fractional_area_index, x[fractional_area_index])    
    else:
        print('No area detected')
        return (None, None) 

- ERN

In [ ]:
roi = ['Fz']
tmin = -0.05
tmax = 0.2
threshold = 1*1e-6

In [ ]:
epochs_df_copy = pd.DataFrame(copy.deepcopy(epochs_df.to_dict()))

In [ ]:
X = Pipeline([
    ("channels_extraction",PickChannels(channels_list=roi)),
    ("trim", EpochTrim(tmin=tmin, tmax=tmax)),
    ("average", Evoked(condition='error_response')),
]).fit_transform(epochs_df_copy)

X = X[['evoked']].to_numpy()

In [ ]:
fractional_latencies = []
for i in range(0, len(X)):        
    subject = X[i][0]
    evoked = subject
    this_latency = fractional_negative_area_latency(evoked, fraction=0.5, tmin=tmin, tmax=tmax, threshold=threshold)
    fractional_latencies.append(this_latency)

In [ ]:
fractional_latencies_ern = list(map(lambda x: x[1] ,fractional_latencies))
fractional_latencies_ern = np.array(fractional_latencies_ern).reshape(-1,1)
fractional_latencies_ern.shape

- CRN: threshold at $2 \mu V$

In [ ]:
roi = ['Fz']
tmin = -0.05
tmax = 0.2
threshold = 2*1e-6 # threshold at 2 uV

In [ ]:
epochs_df_copy = pd.DataFrame(copy.deepcopy(epochs_df.to_dict()))

In [ ]:
X = Pipeline([
    ("channels_extraction",PickChannels(channels_list=roi)),
    ("trim", EpochTrim(tmin=tmin, tmax=tmax)),
    ("average", Evoked(condition='correct_response')),
]).fit_transform(epochs_df_copy)

X = X[['evoked']].to_numpy()

In [ ]:
fractional_latencies = []
for i in range(0, len(X)):        
    subject = X[i][0]
    evoked = subject
    # print(f"Index: {i}")
    this_latency = fractional_negative_area_latency(evoked, fraction=0.5, tmin=tmin, tmax=tmax, threshold=threshold)
    fractional_latencies.append(this_latency)

In [ ]:
fractional_latencies_crn_2uV = list(map(lambda x: x[1] ,fractional_latencies))
fractional_latencies_crn_2uV = np.array(fractional_latencies_crn_2uV).reshape(-1,1)
fractional_latencies_crn_2uV.shape

#### Extract questionnaires scores

Define questionnaires to include in analysis

In [ ]:
rumination = "RRQ"
dass_stress = "DASS-21 Stress"
dass_dep = "DASS-21 Dep"
stai_t = "STAI-T" 
stai_s_diff = "STAI-S Diff" 
uninhibited_responses = "uninhibited response"
inhibited_responses = "inhibited response"
bis = "BIS"
bas_dzialanie = "BAS_D"
bas_przyjemnosc = "BAS_PRZY"
bas_nagroda = "BAS_NAG"
washing = "WASH"
obsessing = "OBSESS"
hoarding = "HOARD"
ordering = "ORD"
checking = "CHECK"
neutralizing = "NEU"
oci_r_full = "OCI-R"
threat = "OT"
thought_suppression = "WBSI"
indecisivness = "INDEC_F"
IU_prospecitve = "IUS-P"
IU_inhibitory = "IUS-I"
self_esteem = "SES"
punishment_sensitivity = "PUN"
reward_sensitivity = "REW"
harm_responsibility = "HARM"
thought_control = "T-CTR"
perfectionism_IU = "OB_PERF"
perfectionism_ps = "PS"
guilt_sensitivity = "G_SE"
intolerance_ambiguity = "AMB"
predictability = "PRED"
high_standards = "STAND"

In [ ]:
scales = [
    rumination,
    dass_stress,
    dass_dep,
    stai_t,
    stai_s_diff,
    uninhibited_responses,
    inhibited_responses,
    bis,
    bas_dzialanie,
    bas_przyjemnosc,
    bas_nagroda,
    washing,
    obsessing,
    hoarding,
    ordering,
    checking,
    neutralizing,
    threat,
    thought_suppression,
    indecisivness,
    punishment_sensitivity,
    reward_sensitivity,
    harm_responsibility,
    guilt_sensitivity,
    thought_control,
    perfectionism_IU,
    perfectionism_ps,
    intolerance_ambiguity,
    predictability,
    high_standards,
    IU_prospecitve,
    IU_inhibitory,
    self_esteem,
]

In [ ]:
questionnaires_scores_df = epochs_df[scales]
questionnaires_scores_df

Fill missing value from external file - TODO to automatisation

In [ ]:
questionnaires_scores_df[questionnaires_scores_df.isna().any(axis=1)]

In [ ]:
if test:
    questionnaires_scores_df.at[102, 'uninhibited response'] = 14.0
    questionnaires_scores_df.at[102, 'inhibited response'] = 98.0
else:
    print('None to fill')

In [ ]:
questionnaires_scores_df[questionnaires_scores_df.isna().any(axis=1)]

Create performance metric based on inhibited and uninhibited responses

In [ ]:
questionnaires_scores_df['performance'] = questionnaires_scores_df['inhibited response'] / questionnaires_scores_df['uninhibited response']
questionnaires_scores_df = questionnaires_scores_df.drop(columns=['inhibited response', 'uninhibited response'])

Inspect data

In [ ]:
questionnaires_scores_df[questionnaires_scores_df.isna().any(axis=1)]

In [ ]:
with pd.option_context('display.max_colwidth', None, 'display.max_columns', None):
    display(questionnaires_scores_df.describe())

Check skewness and kurtiosis

As a general rule of thumb: If skewness is less than -1 or greater than 1, the distribution is highly skewed. If skewness is between -1 and -0.5 or between 0.5 and 1, the distribution is moderately skewed. If skewness is between -0.5 and 0.5, the distribution is approximately symmetric.

Analogous to the skewness, the general guideline is that if the kurtosis is greater than +2, the distribution is too peaked. Likewise, a kurtosis of less than −2 indicates a distribution that is too flat. When both skewness and kurtosis are close to zero, the pattern of responses is considered a normal distribution 

In [ ]:
summary = questionnaires_scores_df.agg(['skew', 'kurtosis', 'mean', 'std', 'min', 'max']).transpose()
summary

Transform entries with high skewness and kurtosis with BoxCox method

In [ ]:
questionnaires_scores_df_transformed = questionnaires_scores_df.copy()

for row in summary.iterrows():
    item_name = row[0]
    skewness_ = row[1]['skew']
    kurtosis_ = row[1]['kurtosis']
    
    if abs(skewness_) > 1 or abs(kurtosis_) > 2:
        print(f'Transforming: {item_name} scale')
        this_scores = questionnaires_scores_df_transformed[[item_name]].to_numpy().flatten()
        this_scores = this_scores - np.min(this_scores) + 0.01
        this_scores_transformed, lambda_ = boxcox(this_scores)
        print(f'   Stats before transformation:\n    Skewness: {scipy.stats.skew(this_scores)} \n    Kurtosis: {scipy.stats.kurtosis(this_scores)}')
        print(f'   Stats after transformation:\n    Skewness: {scipy.stats.skew(this_scores_transformed)} \n    Kurtosis: {scipy.stats.kurtosis(this_scores_transformed)}')
        
        questionnaires_scores_df_transformed[[item_name]] = this_scores_transformed.reshape(-1,1)

In [ ]:
summary = questionnaires_scores_df_transformed.agg(['skew', 'kurtosis', 'mean', 'std', 'min', 'max']).transpose()
summary

#### Demographical data

In [ ]:
age = "Age"
sex = "Sex"
handness = "Handness"

In [ ]:
scales = [
    age,
    sex,
    handness
]

In [ ]:
demographical_scores_df =  epochs_df[scales].astype(float)

In [ ]:
summary = demographical_scores_df.agg(['skew', 'kurtosis', 'mean', 'std', 'min', 'max']).transpose()
summary

Transform skewed age entry

In [ ]:
age = demographical_scores_df[['Age']].to_numpy().flatten()
age_transformed, lambda_ = boxcox(age)
demographical_scores_df[['Age']] = age_transformed.reshape(-1,1)

In [ ]:
summary = demographical_scores_df.agg(['skew', 'kurtosis', 'mean', 'std', 'min', 'max']).transpose()
summary

#### Concatenate questionnaire and EEG features

In [ ]:
eeg_columns_ern = ['e_ERN']
eeg_columns_crn = ['e_CRN']

eeg_column_latencies_fal_2_crn = ['e_LT_F2_C']
eeg_column_latencies_peak_crn = ['e_LT_P_C']
eeg_column_latencies_fal = ['e_LT_F']

1. ERN model

In [ ]:
results_ern_df = pd.DataFrame()

results_ern_df[eeg_columns_ern] = preprocessed_X_ern
results_ern_df.reset_index(drop=True, inplace=True)
results_ern_df.index = epochs_df.index
results_ern_df = pd.concat([results_ern_df, questionnaires_scores_df_transformed.drop(columns=['performance']), epochs_df['id']], axis=1)
results_ern_df.to_pickle(f"../data/models_pickles_new_dass/ern_models_{dataset}_id_clean.pkl")

2. CRN model

In [ ]:
results_crn_df = pd.DataFrame()

results_crn_df[eeg_columns_crn] = preprocessed_X_crn
results_crn_df.reset_index(drop=True, inplace=True)
results_crn_df.index = epochs_df.index
results_crn_df = pd.concat([results_crn_df, questionnaires_scores_df_transformed.drop(columns=['performance']), epochs_df['id']], axis=1)

results_crn_df.to_pickle(f"../data/models_pickles_new_dass/crn_models_{dataset}_id_clean.pkl")

3. ERN and covariates

In [ ]:
results_ern_lat_df = pd.DataFrame()

results_ern_lat_df[eeg_columns_ern] = preprocessed_X_ern
results_ern_lat_df.reset_index(drop=True, inplace=True)
results_ern_lat_df.index = epochs_df.index
results_ern_lat_df[eeg_column_latencies_fal] = fractional_latencies_ern 
results_ern_lat_df = pd.concat([results_ern_lat_df, demographical_scores_df, questionnaires_scores_df_transformed, epochs_df['id']], axis=1)

results_ern_lat_df.to_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_{dataset}_id_clean.pkl")

4. CRN and covariates

In [ ]:
results_crn_lat_demo_df = pd.DataFrame()

results_crn_lat_demo_df[eeg_columns_crn] = preprocessed_X_crn
results_crn_lat_demo_df.reset_index(drop=True, inplace=True)
results_crn_lat_demo_df.index = epochs_df.index
results_crn_lat_demo_df[eeg_column_latencies_fal_2_crn] = fractional_latencies_crn_2uV
results_crn_lat_demo_df = pd.concat([results_crn_lat_demo_df, demographical_scores_df, questionnaires_scores_df_transformed, epochs_df['id']], axis=1)

results_crn_lat_demo_df.to_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_{dataset}_id_clean.pkl")

5. ERN and CRN model

In [ ]:
results_ern_df = pd.DataFrame()

results_ern_df[eeg_columns_ern] = preprocessed_X_ern
results_ern_df[eeg_columns_crn] = preprocessed_X_crn
results_ern_df.reset_index(drop=True, inplace=True)
results_ern_df.index = epochs_df.index
results_ern_df = pd.concat([results_ern_df, questionnaires_scores_df_transformed.drop(columns=['performance']), epochs_df['id']], axis=1)
results_ern_df.to_pickle(f"../data/models_pickles_new_dass/ern_crn_models_{dataset}_id_clean.pkl")

6. ERN, CRN and covariates

In [ ]:
results_ern_lat_df = pd.DataFrame()

results_ern_lat_df[eeg_columns_ern] = preprocessed_X_ern
results_ern_lat_df[eeg_columns_crn] = preprocessed_X_crn

results_ern_lat_df.reset_index(drop=True, inplace=True)
results_ern_lat_df.index = epochs_df.index
results_ern_lat_df[eeg_column_latencies_fal] = fractional_latencies_ern 
results_ern_lat_df[eeg_column_latencies_fal_2_crn] = fractional_latencies_crn_2uV

results_ern_lat_df = pd.concat([results_ern_lat_df, demographical_scores_df, questionnaires_scores_df_transformed, epochs_df['id']], axis=1)
results_ern_lat_df.to_pickle(f"../data/models_pickles_new_dass/ern_crn_cov_fal_models_{dataset}_id_clean.pkl")

## Save datasets

In [ ]:
dataset = 'test'

In [ ]:
ern_data_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_models_{dataset}_id_clean.pkl")
ern_cov_fal_data_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_{dataset}_id_clean.pkl")

crn_data_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_models_{dataset}_id_clean.pkl")
crn_cov_fal2_data_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_{dataset}_id_clean.pkl")

In [ ]:
mapping = {
    "RRQ": "Rumination",
    "DASS-21 Stress": "Stress",
    "DASS-21 Dep": "Depression",
    "STAI-T": "Anxiety trait",
    "STAI-S Diff": 'Affective load',
    "BIS": "Behavioral inhibition",
    "OBSESS": "Obsessing",
    "HOARD": "Hoarding",
    "ORD": "Ordering",
    "CHECK": "Checking",
    'WASH': "Washing",
    'NEU': "Neutralizing",
    "WBSI": "Thought supression",
    "IUS-P": "Prospective IU",
    "IUS-I": "Inhibitiory IU",
    "SES": "Self-esteem",
    'BAS_D': "Drive BAS",
    'BAS_PRZY': "Fun-seeking BAS",
    'BAS_NAG': "Reward responsiveness BAS ",
    'INDEC_F': "Indecisiveness",
    'PUN': "Punishment sensitivity",
    'REW': "Reward sensitivity",
    'HARM': "Inflated harm responsibility",
    'T-CTR': "Importance of thought control",
    "OT": "Threat overestimation",
    'OB_PERF': "Perfectionism/IU",
    'PS': "Personal standandards",
    'G_SE': "Guilt sensitivity",
    'AMB': "Avoidance of ambiguity",
    'PRED': "Need for predictability",
    'STAND': "High standards",   
    "Age": "Age",
    "Handness": "Handedness",
    'e_ERN': "ERN amplitude",
    'e_LT_F': "ERN latency",
    'performance': "Performance",
    'e_CRN': "CRN amplitude",
    'e_LT_F2_C': "CRN latency",
}

In [ ]:
data_df = pd.merge(ern_cov_fal_data_df, crn_cov_fal2_data_df, how='inner')
data_df = data_df.rename(columns=mapping)

In [ ]:
data_df.columns

In [ ]:
data_df.to_csv(f'../public_data/data_{dataset}.csv')